In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
dataset = pd.read_csv("data.csv")

In [3]:
def same_columns(x,y,power):
    count = 0
    dataset[x] = dataset[x].replace(np.nan , 'x')
    dataset[y] = dataset[y].replace(np.nan , 'x')
    for i in tqdm(range(len(dataset))):
        if dataset[x][i] == 'x' and dataset[y][i] != 'x':
            if power == 1 and dataset[y][i]<=7:
                dataset[x][i] = dataset[y][i]
            elif power!=1:
                dataset[x][i] = dataset[y][i]
            else:
                pass
        elif dataset[x][i] == 'x' and dataset[y][i] == 'x':
            count += 1
        else:
            pass
    dataset[x] = dataset[x].replace('x' , np.nan)
    dataset[y] = dataset[y].replace('x' , np.nan)
    return count

In [4]:
c=same_columns("type_of_shot" , "type_of_combined_shot" , 0)
c

  0%|                                                   | 0/30697 [00:00<?, ?it/s]c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
100%|███████████████████████████████████████| 30697/30697 [11:57<00:00, 42.79it/s]


0

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train["game_season"] = le.fit_transform(df_train.iloc[:,7].values.astype(str))

one_hot = pd.get_dummies(df_train["area_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("area_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_train["range_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("range_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_train["shot_basics"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("shot_basics" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_train["home/away"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("home/away" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_train["type_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("type_of_shot" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_train["lat/lng"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("lat/lng" , axis=1,inplace=True)

df_train["shot_id_number"].interpolate(method="linear" , ax=1 , inplace=True)
df_train.drop("date_of_game" , axis=1 , inplace=True)
df_train.drop("shot_id_number" ,axis=1 ,inplace=True)
df_train.drop("type_of_combined_shot" , axis=1 , inplace=True)
#df_train.drop("type_of_combined_shot" ,axis=1 ,inplace=True)

In [4]:
df_train.drop("team_id" , axis=1,inplace=True)
df_train.drop("team_name" , axis=1 ,inplace=True)

In [5]:
len(df_train.columns)

198

In [6]:
bool_series = df_train["is_goal"].isna()
testdata = df_train[bool_series]
false_series = df_train["is_goal"].notnull()
training_data = df_train[false_series]

In [7]:
x = training_data.iloc[: , df_train.columns != "is_goal"].values
y = training_data.iloc[: , df_train.columns == "is_goal"].values
X = testdata.iloc[: , df_train.columns != "is_goal"].values

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)

In [9]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=2 ,learning_rate=0.05)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test , y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 63.40%


In [11]:
len(testdata.columns)

198

In [12]:
y_pred = model.predict_proba(X)
temp = pd.read_csv("cleaned_test_data")
temp["shot_id_number"] = temp["shot_id_number"].astype('int64')
result = pd.DataFrame(data={"shot_id_number" : temp["shot_id_number"], "is_goal" : y_pred[:, 1]})
result

,shot_id_number,is_goal
0,1,0.401387
1,8,0.530493
2,17,0.598729
3,20,0.690176
4,22,0.397563
5,33,0.401387
6,34,0.409460
7,35,0.456960
8,36,0.429457
9,37,0.598234


In [13]:
result.to_csv("2ndAttempt.csv", index=False)